In [2]:
# Map relative path to feature extraction 
import os
import sys
sys.path.append(os.path.join('..','real_time'))

In [3]:
# Import dependencies
import pandas as pd
import plotly.express as px
from obspy import read, UTCDateTime, Stream
from obspy.clients.fdsn import Client
from obspy.io.sac.saczp import _write_sacpz
from functions.pb_utils_v16 import st_FV



In [4]:
# Initialize Waveform Client
client = Client("IRIS")

In [5]:
# Load phase data
emp_file = os.path.join('..','PNSN_metadata','AQMS_Queries','Event_Mag_Phase',
                        'AQMS_event_mag_phase_query_output.csv')
df = pd.read_csv(emp_file)
display(df.columns)
# Convert epoch times into pandas DateTime objects for easier slicing
df.datetime = df.datetime.apply(lambda x: pd.Timestamp(x, unit='s'))
df.arrdatetime = df.arrdatetime.apply(lambda x: pd.Timestamp(x, unit='s'))



Index(['evid', 'magid', 'magnitude', 'magtype', 'uncertainty', 'nsta', 'nobs',
       'orid', 'lat', 'lon', 'depth', 'datetime', 'erhor', 'erlat', 'erlon',
       'sdep', 'gap', 'quality', 'ndef', 'nbs', 'nbfm', 'fdepth', 'fepi',
       'arid', 'arrdatetime', 'iphase', 'sta', 'net', 'seedchan', 'location',
       'fm', 'qual', 'arrquality', 'snr', 'delta', 'seaz', 'in_wgt', 'timeres',
       'rflag'],
      dtype='object')

In [6]:
px.histogram(df,x='arrquality')

In [7]:
# Use the first 10 arrivals with arrquality = 1
df2 = df.copy()[df.arrquality == 1].iloc[:100:10,:]
display(df2)
px.histogram(df2,x='magnitude')

,evid,magid,magnitude,magtype,uncertainty,nsta,nobs,orid,lat,lon,...,location,fm,qual,arrquality,snr,delta,seaz,in_wgt,timeres,rflag
0,61038492,2478093,1.84,l,0.204,15,31,2168443,47.360167,-122.471167,...,,d.,i,1.0,NaN,7.1,9.9,1.0,-0.35,H
40,61037147,2477963,4.02,l,0.152,16,46,2176468,44.089500,-122.831000,...,,d.,i,1.0,NaN,17.2,314.1,1.0,-0.08,H
64,61037172,2589473,1.70,l,0.193,9,16,2689088,47.245000,-123.649167,...,,d.,i,1.0,NaN,32.3,337.7,1.0,-0.31,H
101,61251512,2606093,2.08,l,0.176,22,43,2705048,47.856333,-122.510333,...,,d.,i,1.0,NaN,32.0,167.5,1.0,-0.44,H
137,61041487,2478298,0.70,l,0.219,10,15,2170958,46.313500,-122.170000,...,,d.,i,1.0,NaN,13.2,266.2,1.0,0.18,H
159,61066052,2479893,1.79,l,0.141,4,7,2584143,48.287500,-124.963833,...,,d.,i,1.0,NaN,25.1,87.2,1.0,-0.46,H
199,61037157,2589463,0.85,l,0.144,9,13,2689083,46.431500,-122.388500,...,,c.,i,1.0,NaN,14.5,165.7,1.0,0.13,H
238,61044857,2478613,1.80,l,0.097,16,29,2174398,48.485667,-122.759000,...,,c.,i,1.0,NaN,56.4,209.4,1.0,0.47,H
278,61073391,2480558,0.71,l,0.094,6,8,2190663,46.295500,-122.632333,...,,c.,i,1.0,NaN,18.5,207.6,1.0,-0.14,H
314,61278712,2639448,1.23,l,0.165,9,14,2736863,46.261000,-123.125833,...,,d.,i,1.0,NaN,21.7,155.9,1.0,-0.06,H


In [8]:
holder = []
pp_padding = 1.0
for _I in range(len(df2)):
    # Subset arrival line as a series
    _series = df2.iloc[_I, :]
    # Get Network and Station Code
    gwfk = dict(zip(["network", "station"], _series[["net", "sta"]].values))
    # Handle location formatting differences between AQMS and ObsPy
    if _series.location == "  ":
        gwfk.update({"location": ""})
    else:
        gwfk.update({"location": _series.location})
    # Get channel code with component wildcard `?`
    gwfk.update({"channel": _series.seedchan[:2] + "?"})

    # Get time bounds (start with 7 sec lead, 3 sec lag per Lana et al., 2023)
    tarr = UTCDateTime(_series.arrdatetime.isoformat())
    gwfk.update({"starttime": tarr - 7 - pp_padding})
    gwfk.update({"endtime": tarr + 3 + pp_padding})

    # We will be removing instrument response, so attach instrument RESP with query
    gwfk.update({"attach_response": True})

    ## RUN ACTUAL WAVEFORM QUERY ##
    st = client.get_waveforms(**gwfk)
    # Order stream as ENZ
    st = st.sort()
    # Conduct preprocessing
    # MAKE 2 VERSIONS OF THE TRACE
    # 1) DO EXACTLY THE SAME PREOCESSING
    st_L23 = st.copy()
    # write PAZ file to temp to facilitate exact processing workflow
    _tmp_PAZ_file = os.path.join(".", "_tmp.pz")
    _write_sacpz(
        st_L23[0].stats.response,
        _tmp_PAZ_file
    )
    FV_op = st_FV(
        st_L23, _tmp_
    )
    # 2) test alternative processing workflow
    st_E23 = st.copy()
    # - detrend: 1st order polynomial
    # - taper
    # - filter
    # - resample
    # - detrend again (check this...)
    # - remove instrument response
    # - ANOTHER 1st order polynomial detrend...?
    # - taper
    # - filter

    # Pass preprocessed stream to feature extraction
    # - Extract rectilinearity features
    # - write 3C features to holder
    # For each channel
    # - extract temporal features
    # - extract spectral features
    # - extract cepstral features
    # - write 1C features to holer

# append contents of holder to df2

In [24]:
st[0].stats
RESP = st[0].stats.response
sacpaz = RESP.get_sacpz()
_tr = st[0]
from obspy.io.sac.sacpz import _write_sacpz
attach_paz(_tr, )

Signature: _tr.attach_response(inventories)
Docstring:
Search for and attach channel response to the trace as
:class:`obspy.core.trace.Trace`.stats.response. Raises an exception
if no matching response can be found.
To subsequently deconvolve the instrument response use
:meth:`obspy.core.trace.Trace.remove_response`.

>>> from obspy import read, read_inventory
>>> st = read()
>>> tr = st[0]
>>> inv = read_inventory()
>>> tr.attach_response(inv)
>>> print(tr.stats.response)                  # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
Channel Response
   From M/S (Velocity in Meters per Second) to COUNTS (Digital Counts)
   Overall Sensitivity: 2.5168e+09 defined at 0.020 Hz
   4 stages:
      Stage 1: PolesZerosResponseStage from M/S to V, gain: 1500
      Stage 2: CoefficientsTypeResponseStage from V to COUNTS, ...
      Stage 3: FIRResponseStage from COUNTS to COUNTS, gain: 1
      Stage 4: FIRResponseStage from COUNTS to COUNTS, gain: 1

:type inventories: :class:`~obspy.core.inventor